In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("group_9.csv")

In [3]:
print("Dataset dimension:", df.shape)
display(df.head())

Dataset dimension: (3000, 49)


,duration_1,duration_2,duration_3,duration_4,duration_5,loudness_level,popularity_level,tempo_class,time_signature,key_mode,...,is_instrumental,is_dance_hit,temp_zscore,resonance_factor,timbre_index,echo_constant,distorted_movement,signal_power,target_class,target_regression
0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,0.221824,-0.949100,...,0.0,0.0,-0.128149,-1.114081,0.183527,1,-0.292728,0.972,class_7,0.796292
1,0.0,1.0,0.0,0.0,0.0,3.0,2.0,1.0,-2.089669,1.020640,...,0.0,0.0,0.696314,-0.630931,0.857540,1,-0.154433,0.952,class_7,1.110125
2,0.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.221824,0.204605,...,0.0,0.0,-0.438474,-0.942060,0.198153,1,-0.575080,0.913,class_7,0.616958
3,1.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.221824,-0.076786,...,0.0,0.0,-1.162765,-0.547996,0.272156,1,-0.165958,0.466,class_7,0.347959
4,0.0,1.0,0.0,0.0,0.0,3.0,3.0,1.0,-2.089669,-1.511882,...,0.0,0.0,0.025929,-0.214150,0.523902,1,-1.669916,0.905,class_7,0.930791


Graphic settings

In [4]:
sns.set(style="whitegrid", context="notebook")
plt.rcParams["figure.figsize"] = (10, 6)

Dataset info

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   duration_1                     3000 non-null   float64
 1   duration_2                     3000 non-null   float64
 2   duration_3                     3000 non-null   float64
 3   duration_4                     3000 non-null   float64
 4   duration_5                     3000 non-null   float64
 5   loudness_level                 3000 non-null   float64
 6   popularity_level               3000 non-null   float64
 7   tempo_class                    3000 non-null   float64
 8   time_signature                 3000 non-null   float64
 9   key_mode                       3000 non-null   float64
 10  artist_song_count              3000 non-null   float64
 11  album_freq                     3000 non-null   float64
 12  movement_index                 3000 non-null   f

DESCRIPTIVE STATISTICS
 Basic statistics + skewness and curtose

In [6]:
num_desc = df.describe().T
num_desc["skew"] = df.select_dtypes(include=[np.number]).skew()
num_desc["kurtosis"] = df.select_dtypes(include=[np.number]).kurtosis()
display(num_desc.head(10))

,count,mean,std,min,25%,50%,75%,max,skew,kurtosis
duration_1,3000.0,0.129667,0.335992,0.000000,0.000000,0.000000,0.000000,1.000000,2.205886,2.867844
duration_2,3000.0,0.239333,0.426748,0.000000,0.000000,0.000000,0.000000,1.000000,1.222458,-0.505934
duration_3,3000.0,0.347667,0.476309,0.000000,0.000000,0.000000,1.000000,1.000000,0.640068,-1.591375
duration_4,3000.0,0.236333,0.424900,0.000000,0.000000,0.000000,0.000000,1.000000,1.241904,-0.457980
duration_5,3000.0,0.047000,0.211674,0.000000,0.000000,0.000000,0.000000,1.000000,4.283019,16.355159
loudness_level,3000.0,1.646000,1.333145,0.000000,0.000000,2.000000,3.000000,4.000000,0.049514,-1.488613
popularity_level,3000.0,1.730667,1.047772,0.000000,1.000000,1.000000,3.000000,4.000000,0.643058,-1.067581
tempo_class,3000.0,1.016667,0.305101,0.000000,1.000000,1.000000,1.000000,3.000000,0.564254,8.577322
time_signature,3000.0,0.063872,0.838877,-6.712656,0.221824,0.221824,0.221824,2.533318,-4.879960,34.262170
key_mode,3000.0,0.029570,1.005989,-1.511882,-0.920961,0.176466,1.020640,1.611562,-0.020314,-1.289904
